In [ ]:
%pylab inline

In [ ]:
import seaborn as sns; sns.set()

In [ ]:
from ml import np

In [ ]:
from ml.sampling import mcmc

# Loading the 1D Ising data

In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("data/ising_binary_1d.csv", skiprows=1)

In [ ]:
data = df.values.astype(np.float64)

In [ ]:
train_data, test_data = train_test_split(data)

# RBM

In [ ]:
from ml.rbms import BernoulliRBM

In [ ]:
# RBM parameters
num_visible = 6
num_hidden = 6

In [ ]:
rbm = BernoulliRBM(num_visible, num_hidden)

## Using MCMC kernels

In [ ]:
kernel = mcmc.BlockGibbsKernel(
    [lambda x: rbm.sample_visible(x[0]), lambda x: rbm.sample_hidden(x[0])],
    [0, 1],
    names=["visibles", "hiddens"]
)

In [ ]:
BATCH_SIZE = 4
v_0 = train_data[:BATCH_SIZE]

In [ ]:
initial_state = [v_0, None]

In [ ]:
# sample initial `h`
[v, h] = kernel.sample_group(initial_state, 1)

In [ ]:
kernel.sample([v, h])

### New CD-k

In [ ]:
v_0, h_0 = kernel.sample_group([v_0, None], 1)

sampler = mcmc.Sampler(kernel=kernel, show_progress=False)
trace = sampler.run(initial=[v_0, h_0])

In [ ]:
v_0.shape

In [ ]:
trace[0][0].shape

In [ ]:
trace = np.array(trace)

In [ ]:
trace.shape

In [ ]:
plt.hist(trace[:, 0, :, 0].reshape(-1))

In [ ]:
def contrastive_divergence(v_0, k=1, beta=1.0):
    # sample initial h
    v_0, h_0 = kernel.sample_group([v_0, None], 1)
    [v, h] = [v_0, h_0]

    for _ in range(k):
        [v, h] = kernel.sample([v, h])

    return v_0, h_0, v, h

In [ ]:
rbm.contrastive_divergence(v_0)

In [ ]:
v_0, h_0, v_k, h_k = contrastive_divergence(v_0)
v_0, h_0, v_k, h_k

## Train

In [ ]:
# training parameters
LR = 0.001
BATCH_SIZE = 4096
NUM_EPOCHS = 10
K = 1

V_SIGMA = 0.1